In [2]:
import uuid
from kubeflow import fairing

# docker hub username
CONTAINER_REGISTRY = 'seokii'

# kubectl get namespace로 확인가능
namespace = 'kubeflow-user-example-com'

# job이름 지정
job_name = f'mnist-job-{uuid.uuid4().hex[:4]}'

# command 정의
command=["python", "train.py"]

# output_map 정의
output_map = {
    "dockerfile": "dockerfile",
    "train.py": "train.py"
}

# 페어링 전처리기 정의
fairing.config.set_preprocessor('python', command=command, path_prefix="/app", output_map=output_map)

# cluster builder ContextSource를 minio로 정의
s3_endpoint = 'minio-service.kubeflow.svc.cluster.local:9000'
minio_endpoint = "http://"+s3_endpoint
minio_username = "minio"
minio_key = "minio123"
minio_region = "us-east-1"

# minio_context사용
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource
minio_context_source = MinioContextSource(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)

# 페어링 빌더 정의
fairing.config.set_builder('cluster', registry=CONTAINER_REGISTRY, image_name="train", dockerfile_path="dockerfile",
                           context_source=minio_context_source)

# 페어링 배포 정의
fairing.config.set_deployer('job', namespace=namespace, job_name=job_name, cleanup=False, stream_log=True)

# 페어링 실행
fairing.config.run()

[I 230107 08:24:14 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.base.BasePreProcessor object at 0x7fc67416e130>
[I 230107 08:24:14 config:136] Using builder: <kubeflow.fairing.builders.cluster.cluster.ClusterBuilder object at 0x7fc67419e4f0>
[I 230107 08:24:14 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7fc67419e2e0>
[I 230107 08:24:14 cluster:46] Building image using cluster builder.
[I 230107 08:24:14 base:107] Creating docker context: /tmp/fairing_context_f4w0023b
[W 230107 08:24:15 manager:298] Waiting for fairing-builder-zdq7g-tnjkj to start...
[W 230107 08:24:15 manager:298] Waiting for fairing-builder-zdq7g-tnjkj to start...
[W 230107 08:24:15 manager:298] Waiting for fairing-builder-zdq7g-tnjkj to start...
[I 230107 08:24:16 manager:303] Pod started running True


INFO[0001] Retrieving image manifest tensorflow/tensorflow:2.1.0-py3
INFO[0003] Retrieving image manifest tensorflow/tensorflow:2.1.0-py3
INFO[0005] Built cross stage deps: map[]
INFO[0005] Retrieving image manifest tensorflow/tensorflow:2.1.0-py3
INFO[0006] Retrieving image manifest tensorflow/tensorflow:2.1.0-py3
INFO[0007] Executing 0 build triggers
INFO[0007] Unpacking rootfs as cmd RUN mkdir -p /app requires it.
INFO[0115] Taking snapshot of full filesystem...
INFO[0115] Resolving 28020 paths
INFO[0117] RUN mkdir -p /app
INFO[0117] cmd: /bin/sh
INFO[0117] args: [-c mkdir -p /app]
INFO[0117] Running: [/bin/sh -c mkdir -p /app]
INFO[0117] Using files from context: [/kaniko/buildcontext/train.py]
INFO[0117] ADD train.py /app/
INFO[0117] Taking snapshot of full filesystem...
INFO[0117] Resolving 28022 paths


[W 230107 08:26:20 job:101] The job mnist-job-b53b launched.
[W 230107 08:26:21 manager:298] Waiting for mnist-job-b53b-gcpx8 to start...
[W 230107 08:26:21 manager:298] Waiting for mnist-job-b53b-gcpx8 to start...
[W 230107 08:26:21 manager:298] Waiting for mnist-job-b53b-gcpx8 to start...
[I 230107 08:28:17 manager:303] Pod started running True


2023-01-07 08:28:17.562499: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2023-01-07 08:28:17.562556: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2023-01-07 08:28:17.562561: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
TensorFlow version:  2.1.0
11493376/11490434 [==============================] - 1s 0us/step
2023-01-07 08:28:19.599151: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object f

(<kubeflow.fairing.preprocessors.base.BasePreProcessor at 0x7fc67416e130>,
 <kubeflow.fairing.deployers.job.job.Job at 0x7fc67419e2e0>)